# Video Processing for BT Lab
Video files to png
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [12]:
#import libraries
import os
import re
import cv2
import json
import pandas as pd
import numpy as np

In [13]:
user_drive = input("Enter user drive: ")
video_path = f"{user_drive}:/Christian/DI_centre_structured"
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [14]:
def run_video_to_frame(video_path:str , save_folder:str , old_fps:int, vid_length: float, list_of_frames:list[str]):
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()
    pick_counter = 0
    save_counter = 0
    set_counter = 1
    true_frames = 0
    success = True

    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)

    print(f"saving video to {save_folder},\npicking frames {list_of_frames}\nexpecting {vid_length*len(list_of_frames)} png files")

    while success and set_counter <= vid_length:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            frame_name = "frame_%d.png" % true_frames
            frame_path = os.path.join(save_folder, frame_name)
            cv2.imwrite(frame_path, frame) 
            save_counter += 1

        pick_counter += 1
        true_frames += 1

        if (pick_counter == old_fps):
            set_counter += 1
            pick_counter = 0

    video.release()
    return [set_counter, save_counter, true_frames]

In [15]:
def get_frames_path(local_path: str, level:str) -> str:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return folder_path

In [16]:
def down_sample(old_fps, new_fps):
    down_sampled_list = []
    user_frames = np.arange(0, old_fps, dtype=int)
    step = old_fps / new_fps

    for fps in range(new_fps):
        start = int(fps * step)
        end = int((fps + 1) * step)
        
        if fps == new_fps - 1:
            end = old_fps # ensures that the last element is included
        
        avg = sum(user_frames[start:end]) // (end - start)
        down_sampled_list.append(avg)

    return down_sampled_list

In [17]:
def up_sample(old_fps, new_fps):
    pass

In [18]:
def convert_video_to_frame(all_patients:dict, rgb_fps: dict):
    for patient_id, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])
            vid_length = int(patient_info["length"])

            for level, new_fps in rgb_fps.items():
                frames_folder = get_frames_path(video_path, level)
                
                if old_fps <= new_fps:
                    frames_to_pick = up_sample(old_fps, new_fps)
                else:
                    frames_to_pick = down_sample(old_fps, new_fps)
                
                if len(frames_to_pick) != new_fps:
                    raise ValueError("Number of frames to pick is not equal to new fps")
                
                set_counter, save_counter, true_frames = run_video_to_frame(video_path, frames_folder, old_fps, vid_length, frames_to_pick)

                print(f"set counter: {set_counter}, save counter: {save_counter}, true frames: {true_frames}")

                break

        except Exception as e:
            print(f"type error: {e} for patient {video_path}")

        break;

In [19]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [20]:
""" local vals"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [21]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_complete.json")
metadata_thermal = load_json(json_dir, "thermal_complete.json")

In [22]:
""" convert video to png (rgb) """
convert_video_to_frame(metadata_rgb, rgb_fps)

D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound 29 [0, 3, 6, 9, 12, 15, 18, 21, 24, 27]


set counter: 11, save counter: 100, true frames: 290
